In [17]:
#!pip install mne scipy
#!pip install pandas numpy openpyxl
#!pip install tsfresh
#!pip install PyWavelets

In [18]:
import os
import numpy as np
import scipy.signal as signal
import mne

def process_all_eeg_data() -> dict:
    """
    Process all .bdf EEG files in the current directory, applying filters and extracting data from
    channels A15 (O1), A16 (Oz), and A17 (O2).

    Returns
    -------
    dict
        A dictionary containing processed EEG data and header information for each file.
    """
    # Get a list of all .bdf files in the current directory
    files = [f for f in os.listdir('.') if f.endswith('.bdf')]
    if not files:
        raise FileNotFoundError("No BDF files found in the current directory")
    
    # Initialize the results dictionary
    results = {}
    
    # Loop over each file
    for filename in files:
        full_file_path = os.path.join(os.getcwd(), filename)
        
        # Read the raw EEG data using MNE
        raw = mne.io.read_raw_bdf(full_file_path, preload=True)
        hdr = raw.info
        
        # Select data from channels A15 (O1), A16 (Oz), and A17 (O2)
        channels_select = ['A15', 'A16', 'A17']
        missing_channels = [ch for ch in channels_select if ch not in hdr['ch_names']]
        if missing_channels:
            raise ValueError(f"Selected channels {missing_channels} not found in the data")
        
        channel_indices = [hdr['ch_names'].index(ch) for ch in channels_select]
        EEG_data = raw.get_data(picks=channel_indices).T  # Shape: (n_samples, n_channels)
        
        # Filter EEG Data
        Fs = hdr['sfreq']  # Sampling frequency
        
        # Bandpass filter parameters (2 to 80 Hz)
        Fc_BP = [2, 80]  # Bandpass frequency range
        Wn_BP = [f / (Fs / 2) for f in Fc_BP]  # Normalize by Nyquist frequency
        
        # Create and apply bandpass filter (6th order zero-phase Butterworth IIR)
        B_BP, A_BP = signal.butter(3, Wn_BP, btype='bandpass')
        EEG_filtered_BP = signal.filtfilt(B_BP, A_BP, EEG_data, axis=0)
        
        # Band stop filter parameters (48 to 52 Hz)
        Fc_BS = [48, 52]  # Band stop frequency range
        Wn_BS = [f / (Fs / 2) for f in Fc_BS]  # Normalize by Nyquist frequency
        
        # Create and apply band stop filter (6th order zero-phase Butterworth IIR)
        B_BS, A_BS = signal.butter(3, Wn_BS, btype='bandstop')
        EEG_filtered = signal.filtfilt(B_BS, A_BS, EEG_filtered_BP, axis=0)
        
        # Extract prefix before underscore from the filename
        underscore_index = filename.find('_')
        if underscore_index == -1:
            raise ValueError(f"Filename format error, no underscore found in {filename}")
        key = filename[:underscore_index]
        
        # Store results in the dictionary
        results[key] = {
            'data': EEG_filtered,      # Filtered data for channels A15, A16, A17
            'channels': channels_select,  # List of channel names
            'header': hdr
        }
        
        # Display a message indicating successful processing
        print(f"Data for file {filename} processed successfully")
    
    return results


In [19]:
results = process_all_eeg_data()

Extracting EDF parameters from c:\Users\WERPELGA\OneDrive - Danone\Desktop\UoA\2024.1&2\Python Gabe\A1_Full_Block.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 739327  =      0.000 ...   361.000 secs...
Data for file A1_Full_Block.bdf processed successfully
Extracting EDF parameters from c:\Users\WERPELGA\OneDrive - Danone\Desktop\UoA\2024.1&2\Python Gabe\A3_Full_Block.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 757759  =      0.000 ...   370.000 secs...
Data for file A3_Full_Block.bdf processed successfully
Extracting EDF parameters from c:\Users\WERPELGA\OneDrive - Danone\Desktop\UoA\2024.1&2\Python Gabe\A4_Full_Block.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 782335  =      0.000 ...   382.000 secs...
Data for file A4_Full_Block.bdf processed successfully
Extracting EDF parameters from c:\Users\WERPELGA\One

In [20]:
import numpy as np
import pandas as pd

def segment_eeg_data_new(results: dict, cohort_file: str = 'Cohort.xlsx') -> dict:
    """
    Segments EEG data into predefined sections (EC, EO, LC, RC, DEC, NDEC) based on cohort information,
    removing the first 2 seconds from each section.

    Parameters
    ----------
    results : dict
        Dictionary containing the raw EEG data and header information for each key (participant).
    cohort_file : str, optional
        Path to the Excel file containing cohort information (default is 'Cohort.xlsx').

    Returns
    -------
    dict
        Dictionary containing segmented EEG data for each participant.
    """
    # Read the cohort information from an Excel file
    cohort_table = pd.read_excel(cohort_file)
    # Segment Duration (in seconds)
    segment_duration = 10  # Original segment duration in seconds
    skip_duration = 2      # Duration to skip at the start of each segment (2 seconds)

    # Initialize the segmented results dictionary
    segmented_data = {}

    # Iterate through each key in the results dictionary
    for key, result in results.items():
        data = result['data']  # Data shape: (n_samples, n_channels)
        hdr = result['header']

        # Find the matching row in the cohort table
        cohort_row = cohort_table[cohort_table['Cohort'] == key]
        
        if cohort_row.empty:
            raise ValueError(f"Cohort information not found for {key}")

        # Define the sample rate and calculate sample counts
        Fs = hdr['sfreq']  # Sampling frequency
        samples_per_segment = int(segment_duration * Fs)
        samples_to_skip = int(skip_duration * Fs)
        effective_samples_per_segment = samples_per_segment - samples_to_skip
        n_channels = data.shape[1]  # Number of channels (should be 3: O1, Oz, O2)

        # Initialize segments with zeros
        EC = np.zeros((effective_samples_per_segment, n_channels))
        EO = np.zeros((effective_samples_per_segment, n_channels))
        LC = np.zeros((effective_samples_per_segment, n_channels))
        RC = np.zeros((effective_samples_per_segment, n_channels))
        DEC = np.zeros((effective_samples_per_segment, n_channels))
        NDEC = np.zeros((effective_samples_per_segment, n_channels))

        # Fill segments with data if available, skipping the first 2 seconds
        # EC segment
        segment_start = 0
        segment_end = samples_per_segment
        if data.shape[0] >= segment_end:
            EC = data[segment_start + samples_to_skip : 0, :]
        else:
            print(f"Not enough data for EC segment in {key}")

        # EO segment
        segment_start = samples_per_segment
        segment_end = 2 * samples_per_segment
        if data.shape[0] >= segment_end:
            EO = data[segment_start + samples_to_skip : segment_end, :]
        else:
            print(f"Not enough data for EO segment in {key}")

        # LC segment
        segment_start = 2 * samples_per_segment
        segment_end = 3 * samples_per_segment
        if data.shape[0] >= segment_end:
            LC = data[segment_start + samples_to_skip : segment_end, :]
        else:
            print(f"Not enough data for LC segment in {key}")

        # RC segment
        segment_start = 3 * samples_per_segment
        segment_end = 4 * samples_per_segment
        if data.shape[0] >= segment_end:
            RC = data[segment_start + samples_to_skip : segment_end, :]
        else:
            print(f"Not enough data for RC segment in {key}")

        # Apply conditions based on cohort table
        if cohort_row['LC'].values[0] == 'DEC':
            # Assign 'DEC' to LC and 'NDEC' to RC
            DEC = LC
            NDEC = RC
        elif cohort_row['RC'].values[0] == 'DEC':
            # Assign 'DEC' to RC and 'NDEC' to LC
            DEC = RC
            NDEC = LC
        else:
            # If neither LC nor RC is 'DEC', assign NDEC accordingly
            NDEC = LC
            # Optionally handle cases where DEC is not specified
            DEC = RC  # Or set DEC to zeros if appropriate

        # Store the segmented data and 'LinesDifference' in the results dictionary
        segmented_data[key] = {
            'header': hdr,
            'EC': EC,
            'EO': EO,
            'DEC': DEC,
            'NDEC': NDEC,
            'LinesDifference': cohort_row['LinesDifference'].values[0]
        }

    return segmented_data


In [21]:
segmented_data = segment_eeg_data_new(results)

In [22]:
import pandas as pd
import numpy as np
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

def prepare_time_series_by_section(segmented_data, cohort_table):
    """
    Prepares a DataFrame suitable for tsfresh from segmented EEG data for all sections (EC, EO, DEC, NDEC).

    Parameters
    ----------
    segmented_data : dict
        The dictionary containing segmented EEG data for each participant.
    cohort_table : pd.DataFrame
        DataFrame containing cohort information (including labels for Amblyopia/Control).

    Returns
    -------
    pd.DataFrame, pd.Series
        A DataFrame where each row represents a time-series sample with columns 'id', 'time', 'O1', 'Oz', 'O2',
        and a Series with group labels indexed by 'id'.
    """
    data_list = []
    labels_list = []

    # Loop through each participant's data
    for key, value in segmented_data.items():
        # Find the matching cohort row
        cohort_row = cohort_table[cohort_table['Cohort'] == key]
        if cohort_row.empty:
            continue

        # Assign label based on the first letter of the 'Cohort' column (Amblyopia = 1, Control = 0)
        label = 1 if key.startswith('A') else 0

        # Get channel names; default to ['O1', 'Oz', 'O2'] if not available
        channels = value.get('channels', ['O1', 'Oz', 'O2'])

        # For each section (EC, EO, DEC, NDEC)
        for section in ['EC', 'EO', 'DEC', 'NDEC']:
            section_data = value[section]  # Shape: (n_samples, n_channels)

            # Create a DataFrame for this section
            n_samples = section_data.shape[0]
            df = pd.DataFrame({
                'id': f"{key}_{section}",
                'time': np.arange(n_samples)
            })

            # Add each channel's data as a column
            for idx, channel_name in enumerate(channels):
                df[channel_name] = section_data[:, idx]

            # Append to data list
            data_list.append(df)

            # Append label for this 'id' (participant_section)
            labels_list.append({'id': f"{key}_{section}", 'label': label})

    # Concatenate all data into a single DataFrame
    time_series_df = pd.concat(data_list, ignore_index=True)

    # Create a labels DataFrame and convert to a Series indexed by 'id'
    labels_df = pd.DataFrame(labels_list).drop_duplicates(subset='id')
    labels_series = labels_df.set_index('id')['label']

    # Return the time-series data and corresponding labels
    return time_series_df, labels_series

# Load your cohort table (must include 'Cohort' column)
cohort_table = pd.read_excel('Cohort.xlsx')

# Prepare the time series DataFrame and labels
time_series_df, labels = prepare_time_series_by_section(segmented_data, cohort_table)


In [23]:
time_series_df

,id,time,O1,Oz,O2
0,A1_EO,0,-0.000013,-0.000026,-0.000036
1,A1_EO,1,-0.000016,-0.000028,-0.000038
2,A1_EO,2,-0.000019,-0.000031,-0.000040
3,A1_EO,3,-0.000022,-0.000034,-0.000042
4,A1_EO,4,-0.000025,-0.000036,-0.000043
...,...,...,...,...,...
638971,C1_NDEC,16379,-0.000003,-0.000004,-0.000004
638972,C1_NDEC,16380,-0.000003,-0.000003,-0.000004
638973,C1_NDEC,16381,-0.000003,-0.000002,-0.000003
638974,C1_NDEC,16382,-0.000003,-0.000002,-0.000003


In [24]:
import pandas as pd

# Save time_series_df as CSV
time_series_df.to_csv('time_series_df_full.csv', index=False)

# Save labels as CSV
labels.to_csv('labels_full.csv', index=False, header=True)

# Optionally, save labels as Pickle (preserves Python object types)
# labels.to_pickle('labels.pkl')

In [25]:
# import pandas as pd

# # Read time_series_df from CSV
# time_series_df = pd.read_csv('time_series_df_full.csv')

# # Read labels from CSV
# labels = pd.read_csv('labels_full.csv')



In [26]:
labels.index = time_series_df['id'].unique()

ValueError: Length mismatch: Expected axis has 52 elements, new values have 39 elements

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
import gc
from tsfresh import extract_features
from tsfresh.feature_extraction import ComprehensiveFCParameters, MinimalFCParameters

# Define the function to process data in chunks using ComprehensiveFCParameters
def process_in_chunks(time_series_df, N):
    # Get unique IDs
    unique_ids = time_series_df['id'].unique()
    
    # Split the unique IDs into chunks of size N
    chunks = [unique_ids[i:i + N] for i in range(0, len(unique_ids), N)]
    
    # Initialize an empty list to store the results
    results = []
    
    # Process each chunk
    for chunk in chunks:
        # Filter the DataFrame to include only the IDs in the current chunk
        chunk_df = time_series_df[time_series_df['id'].isin(chunk)]
        
        # Extract features for the current chunk using ComprehensiveFCParameters
        extracted_features_chunk = extract_features(
            chunk_df,
            column_id='id',
            column_sort='time',
            default_fc_parameters=MinimalFCParameters()
        )
        
        # Append the extracted features to the results list
        results.append(extracted_features_chunk)
        
        # Clear memory
        del chunk_df, extracted_features_chunk
        gc.collect()
    
    # Concatenate all the results into a single DataFrame
    final_result = pd.concat(results)
    
    return final_result

# Set the chunk size N (adjust based on your memory constraints)
N = 10  # Smaller chunk size to manage memory usage

# Extract features using the process_in_chunks function
extracted_features = process_in_chunks(time_series_df, N)

# Drop any columns with NaN or infinite values
extracted_features_clean = extracted_features.replace([np.inf, -np.inf], np.nan).dropna(axis=1)

# Ensure that the labels are aligned with the extracted features
# Assuming 'labels' is a Series with 'id' as the index
labels_aligned = labels.loc[extracted_features_clean.index]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    extracted_features_clean,
    labels_aligned,
    test_size=0.3,
    random_state=42
)

# Select the most important features using ANOVA F-test
selector = SelectKBest(f_classif, k=10)  # Adjust 'k' as needed
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

# Train a Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Define parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
}

# Perform grid search to find the best parameters
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_selected, y_train)

# Print the best parameters found by GridSearchCV
print(f"Best parameters: {grid_search.best_params_}")

# Use the best estimator from GridSearchCV to predict and evaluate the model
best_clf = grid_search.best_estimator_
y_pred = best_clf.predict(X_test_selected)

# Evaluate the model
print(classification_report(y_test, y_pred))

# Identify and display the top selected features with importance
selected_feature_names = extracted_features_clean.columns[selector.get_support()]
important_features = pd.DataFrame({
    'Feature': selected_feature_names,
    'Importance': best_clf.feature_importances_
}).sort_values(by='Importance', ascending=False)

print(important_features)


Feature Extraction: 100%|██████████| 14/14 [00:01<00:00,  7.47it/s]
c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [ 3 13 23] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


Best parameters: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 200}
              precision    recall  f1-score   support

           0       0.62      0.83      0.71         6
           1       0.75      0.50      0.60         6

    accuracy                           0.67        12
   macro avg       0.69      0.67      0.66        12
weighted avg       0.69      0.67      0.66        12

                  Feature  Importance
9             O2__minimum    0.160322
0             O1__maximum    0.143272
2          O2__sum_values    0.136311
4  O2__standard_deviation    0.127567
6    O2__root_mean_square    0.124693
1    O1__absolute_maximum    0.104424
7             O2__maximum    0.103111
8    O2__absolute_maximum    0.100300
3                O2__mean    0.000000
5            O2__variance    0.000000


1. Best Parameters Found by GridSearchCV
plaintext
Copy code
Best parameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}
Explanation:

max_depth: None: This means there's no limit to how deep each tree in the forest can grow. The nodes will expand until all leaves are pure or until all leaves contain fewer samples than min_samples_split.

min_samples_split: 2: This is the minimum number of samples required to split an internal node. A value of 2 is the default and allows the tree to grow as much as possible.

n_estimators: 100: The number of trees in the forest is 100. More trees can lead to better performance but also increase computation time.

Interpretation:

The grid search determined that the default parameters are optimal within the range you provided. Essentially, the model performs best without restrictions on tree depth and with the default settings for splitting and the number of trees.

2. Classification Report
plaintext
Copy code
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         6
           1       1.00      0.40      0.57        10

    accuracy                           0.62        16
   macro avg       0.75      0.70      0.62        16
weighted avg       0.81      0.62      0.61        16
Metrics Explanation:

Support: The number of occurrences of each class in the test set.

Class 0: 6 instances.
Class 1: 10 instances.
Precision: The ratio of correctly predicted positive observations to the total predicted positives.

Class 0: 0.50 (50% of the instances predicted as class 0 are actually class 0).
Class 1: 1.00 (100% of the instances predicted as class 1 are actually class 1).
Recall: The ratio of correctly predicted positive observations to all actual positives.

Class 0: 1.00 (100% of actual class 0 instances are correctly identified).
Class 1: 0.40 (Only 40% of actual class 1 instances are correctly identified).
F1-score: The harmonic mean of precision and recall.

Class 0: 0.67.
Class 1: 0.57.
Accuracy: Overall, 62% of the test set instances are correctly classified.

Interpretation:

Class 0 (Control Group or Non-Amblyopia):

High Recall (1.00): The model correctly identified all instances of class 0.
Low Precision (0.50): Half of the instances predicted as class 0 are actually from class 1 (false positives).
Class 1 (Amblyopia Group):

High Precision (1.00): All instances predicted as class 1 are correctly from class 1.
Low Recall (0.40): The model failed to identify 60% of actual class 1 instances (false negatives).
Overall Performance:

The model is better at identifying class 0 but struggles to correctly identify all instances of class 1.
Accuracy is 62%, which may not be satisfactory depending on the context.
Macro Average:
Precision: 0.75.
Recall: 0.70.
F1-score: 0.62.
Possible Reasons for the Performance:

Class Imbalance: Although the classes are relatively balanced (6 vs. 10), the model may still be biased towards class 0.
Small Dataset: With only 16 instances in the test set, the model's performance metrics may not be stable or representative.
Overfitting: The model may have overfitted to the training data, especially if the training set is small or if the model is too complex.
Feature Selection: Limiting to 10 features may have excluded important predictors.
3. Important Features and Their Importances
plaintext
Copy code
                      Feature  Importance
4             O1__maximum    0.207673
8    O2__absolute_maximum    0.140004
7             O2__maximum    0.126442
9             O2__minimum    0.125662
1  O1__standard_deviation    0.092274
5    O1__absolute_maximum    0.088193
6             O1__minimum    0.083645
3    O1__root_mean_square    0.077596
0              O1__median    0.058510
2            O1__variance    0.000000
Feature Descriptions:

Channel O1:

O1__maximum: The maximum value of the EEG signal in the O1 channel.
O1__absolute_maximum: The largest absolute value in the O1 channel.
O1__minimum: The minimum value in the O1 channel.
O1__standard_deviation: The standard deviation of the O1 signal.
O1__root_mean_square: The RMS value of the O1 signal.
O1__median: The median value of the O1 signal.
O1__variance: The variance of the O1 signal.
Channel O2:

O2__absolute_maximum: The largest absolute value in the O2 channel.
O2__maximum: The maximum value of the EEG signal in the O2 channel.
O2__minimum: The minimum value in the O2 channel.
Feature Importances:

The importance values indicate the relative contribution of each feature to the model's decision-making.

Top Features:

O1__maximum (0.2077): Most significant feature.
O2__absolute_maximum (0.1400).
O2__maximum (0.1264).
O2__minimum (0.1257).
Zero Importance Feature:

O1__variance (0.0000): This feature did not contribute to the model's predictions.
Interpretation:

The model heavily relies on maximum and minimum amplitude values from channels O1 and O2.
Features like standard deviation and root mean square also play a role but are less significant.
The variance of the O1 signal didn't contribute, possibly due to redundancy with other features or lack of discriminative power.
4. Overall Analysis and Recommendations
Understanding the Model's Behavior:

High Recall for Class 0: The model correctly identifies all control group instances but at the cost of misclassifying many amblyopia instances as controls.
Low Recall for Class 1: The model misses 60% of the amblyopia cases, which is critical if the goal is to detect amblyopia.
Feature Dependence: The model's reliance on extreme values (maximum and minimum) may make it sensitive to noise or outliers in the data.
Possible Issues:

Data Quality: EEG data can be noisy. Extreme values may be influenced by artifacts rather than true neural activity.
Overfitting to Noise: Focusing on maximum and minimum values might cause the model to capture noise rather than meaningful patterns.
Small Sample Size: With a limited number of samples, especially in the test set, performance metrics may not be reliable.
Feature Selection Limitations: Selecting only 10 features may not capture the complexity needed to differentiate between classes.
Recommendations:

Increase Dataset Size:

Collect more EEG recordings to provide the model with more examples to learn from.
Enhance Feature Extraction:

Use more comprehensive feature extraction methods, possibly including frequency-domain features (e.g., power spectral density).
Consider time-frequency analysis (e.g., wavelet transforms) to capture transient events.
Feature Selection Strategy:

Instead of selecting a fixed number of features (k), consider using all features or use techniques like recursive feature elimination (RFE) to find the optimal feature subset.
Evaluate feature importance using different criteria, such as mutual information.
Address Class Imbalance:

Although the class distribution isn't severely imbalanced, using techniques like SMOTE (Synthetic Minority Over-sampling Technique) can help the model learn better representations of the minority class.
Ensure that the train-test split maintains class distribution (stratified sampling).
Model Tuning and Validation:

Experiment with different models, such as Gradient Boosting Machines, Support Vector Machines, or Neural Networks.
Use cross-validation to obtain a more reliable estimate of the model's performance.
Adjust Evaluation Metrics:

Since missing amblyopia cases is more critical, consider optimizing for recall on class 1.
Use metrics like ROC AUC or Precision-Recall curves to get a better understanding of model performance.
Data Preprocessing:

Apply signal processing techniques to reduce noise, such as filtering or artifact rejection.
Normalize or standardize the features to reduce the impact of scale differences.
Investigate Feature Importances:

Analyze why certain features are more important.
Consider if these features make sense from a neuroscientific perspective.
Conclusion
Model Limitations: The current model doesn't perform adequately, especially in detecting the amblyopia class, which is critical for your application.
Next Steps: Implement the recommendations to improve data quality, feature richness, and model robustness.
Continuous Evaluation: As you make changes, continue to evaluate the model using appropriate metrics and validation strategies.
Remember: Machine learning in healthcare and biomedical applications often requires careful consideration of data quality, feature engineering, and ethical implications of false negatives and false positives. It's crucial to ensure that the model is reliable and performs well on the aspects that matter most for the intended use case.

Let me know if you have any questions or need further clarification on any of these points!




In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
import gc
from tsfresh import extract_features
from tsfresh.feature_extraction import ComprehensiveFCParameters, MinimalFCParameters
from tsfresh.utilities.dataframe_functions import impute

# Define the function to process data in chunks using ComprehensiveFCParameters
def process_in_chunks(time_series_df, N):
    # Get unique IDs
    unique_ids = time_series_df['id'].unique()
    
    # Split the unique IDs into chunks of size N
    chunks = [unique_ids[i:i + N] for i in range(0, len(unique_ids), N)]
    
    # Initialize an empty list to store the results
    results = []
    
    # Process each chunk
    for chunk in chunks:
        # Filter the DataFrame to include only the IDs in the current chunk
        chunk_df = time_series_df[time_series_df['id'].isin(chunk)]
        
        # Extract features for the current chunk using ComprehensiveFCParameters
        extracted_features_chunk = extract_features(
            chunk_df,
            column_id='id',
            column_sort='time',
            default_fc_parameters=MinimalFCParameters(),  # Use ComprehensiveFCParameters() for more features
            n_jobs=4,  # Adjust based on your CPU cores
            # Since data is in wide format, we do not need to specify column_kind and column_value
        )
        
        # Impute missing values in the extracted features
        impute(extracted_features_chunk)
        
        # Append the extracted features to the results list
        results.append(extracted_features_chunk)
        
        # Clear memory
        del chunk_df, extracted_features_chunk
        gc.collect()
    
    # Concatenate all the results into a single DataFrame
    final_result = pd.concat(results)
    
    return final_result

# Set the chunk size N (adjust based on your memory constraints)
N = 10  # Smaller chunk size to manage memory usage

# Extract features using the process_in_chunks function
extracted_features = process_in_chunks(time_series_df, N)

# Drop any columns with NaN or infinite values
extracted_features_clean = extracted_features.replace([np.inf, -np.inf], np.nan).dropna(axis=1)

# Ensure that the labels are aligned with the extracted features
# Assuming 'labels' is a Series with 'id' as the index
labels_aligned = labels.loc[extracted_features_clean.index]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    extracted_features_clean,
    labels_aligned,
    test_size=0.3,
    random_state=42,
    stratify=labels_aligned  # Ensure stratified sampling
)

# Select the most important features using ANOVA F-test
selector = SelectKBest(f_classif, k=10)  # Adjust 'k' as needed
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

# Get the names of the selected features
selected_feature_names = extracted_features_clean.columns[selector.get_support()]

# Train a Random Forest Classifier
clf = RandomForestClassifier(random_state=42)

# Define parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
}

# Perform grid search to find the best parameters
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_selected, y_train)

# Print the best parameters found by GridSearchCV
print(f"Best parameters: {grid_search.best_params_}")

# Use the best estimator from GridSearchCV to predict and evaluate the model
best_clf = grid_search.best_estimator_
y_pred = best_clf.predict(X_test_selected)

# Evaluate the model
print(classification_report(y_test, y_pred))

# Identify and display the top selected features with importance
important_features = pd.DataFrame({
    'Feature': selected_feature_names,
    'Importance': best_clf.feature_importances_
}).sort_values(by='Importance', ascending=False)

print(important_features)


Feature Extraction: 100%|██████████| 14/14 [00:01<00:00,  8.12it/s]
c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [ 3 13 23] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


Best parameters: {'max_depth': None, 'min_samples_split': 10, 'n_estimators': 500}
              precision    recall  f1-score   support

           0       0.67      0.67      0.67         6
           1       0.67      0.67      0.67         6

    accuracy                           0.67        12
   macro avg       0.67      0.67      0.67        12
weighted avg       0.67      0.67      0.67        12

                  Feature  Importance
8    O2__absolute_maximum    0.231114
0          Oz__sum_values    0.165242
7             O2__maximum    0.147457
2          O2__sum_values    0.136516
9             O2__minimum    0.133236
4  O2__standard_deviation    0.104246
6    O2__root_mean_square    0.082190
1                Oz__mean    0.000000
3                O2__mean    0.000000
5            O2__variance    0.000000


In [33]:
!pip install xgboost

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/e2/7b/8c1b410cd0604cee9a167a19f7e1746f5b92ae7d02ad574ab560b73c5a48/xgboost-2.1.1-py3-none-win_amd64.whl.metadata
   --------------------------------------- 124.9/124.9 MB 10.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
# Import necessary libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier  # Import XGBoost classifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import gc
from tsfresh import extract_features
from tsfresh.feature_extraction import ComprehensiveFCParameters, MinimalFCParameters
from tsfresh.utilities.dataframe_functions import impute

# Define the function to process data in chunks using ComprehensiveFCParameters
def process_in_chunks(time_series_df, N):
    # Get unique IDs
    unique_ids = time_series_df['id'].unique()
    
    # Split the unique IDs into chunks of size N
    chunks = [unique_ids[i:i + N] for i in range(0, len(unique_ids), N)]
    
    # Initialize an empty list to store the results
    results = []
    
    # Process each chunk
    for chunk in chunks:
        # Filter the DataFrame to include only the IDs in the current chunk
        chunk_df = time_series_df[time_series_df['id'].isin(chunk)]
        
        # Extract features for the current chunk using ComprehensiveFCParameters
        extracted_features_chunk = extract_features(
            chunk_df,
            column_id='id',
            column_sort='time',
            default_fc_parameters=MinimalFCParameters(),  # Use ComprehensiveFCParameters() for more features
            n_jobs=4,  # Adjust based on your CPU cores
            # Since data is in wide format, we do not need to specify column_kind and column_value
        )
        
        # Impute missing values in the extracted features
        impute(extracted_features_chunk)
        
        # Append the extracted features to the results list
        results.append(extracted_features_chunk)
        
        # Clear memory
        del chunk_df, extracted_features_chunk
        gc.collect()
    
    # Concatenate all the results into a single DataFrame
    final_result = pd.concat(results)
    
    return final_result

# Set the chunk size N (adjust based on your memory constraints)
N = 10  # Smaller chunk size to manage memory usage

# Extract features using the process_in_chunks function
extracted_features = process_in_chunks(time_series_df, N)

# Drop any columns with NaN or infinite values
extracted_features_clean = extracted_features.replace([np.inf, -np.inf], np.nan).dropna(axis=1)

# Ensure that the labels are aligned with the extracted features
# Assuming 'labels' is a Series with 'id' as the index
labels_aligned = labels.loc[extracted_features_clean.index]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    extracted_features_clean,
    labels_aligned,
    test_size=0.3,
    random_state=42,
    stratify=labels_aligned  # Ensure stratified sampling
)

# Select the most important features using ANOVA F-test
selector = SelectKBest(f_classif, k=10)  # Adjust 'k' as needed
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

# Get the names of the selected features
selected_feature_names = extracted_features_clean.columns[selector.get_support()]

# Initialize an empty dictionary to store classifiers and their parameter grids
classifiers = {
    'Random Forest': {
        'model': RandomForestClassifier(random_state=42),
        'param_grid': {
            'n_estimators': [100, 200, 500],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10],
        }
    },
    'Logistic Regression': {
        'model': LogisticRegression(random_state=42, max_iter=5000),
        'param_grid': {
            'penalty': ['l1', 'l2'],
            'C': [0.01, 0.1, 1, 10, 100],
            'solver': ['liblinear'],
        }
    },
    'Support Vector Machine': {
        'model': SVC(random_state=42),
        'param_grid': [
            {'kernel': ['linear'], 'C': [0.1, 1, 10, 100]},
            {'kernel': ['rbf'], 'C': [0.1, 1, 10, 100], 'gamma': ['scale', 'auto']},
            {'kernel': ['poly'], 'C': [0.1, 1, 10], 'degree': [2, 3], 'gamma': ['scale', 'auto']}
        ]
    },
    'Gradient Boosting': {
        'model': GradientBoostingClassifier(random_state=42),
        'param_grid': {
            'n_estimators': [100, 200],
            'learning_rate': [0.01, 0.1],
            'max_depth': [3, 5],
            'min_samples_split': [2, 5],
        }
    },
    'Neural Network': {
        'model': Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', MLPClassifier(random_state=42, max_iter=500))
        ]),
        'param_grid': {
            'classifier__hidden_layer_sizes': [(50,), (100,), (50, 50)],
            'classifier__activation': ['tanh', 'relu'],
            'classifier__solver': ['adam', 'sgd'],
            'classifier__alpha': [0.0001, 0.001],
            'classifier__learning_rate': ['constant', 'adaptive'],
        }
    },
    'XGBoost': {
        'model': XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
        'param_grid': {
            'n_estimators': [100, 200],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1, 0.2],
            'subsample': [0.7, 0.8, 1.0],
            'colsample_bytree': [0.7, 0.8, 1.0],
        }
    }
}

# Loop through each classifier, perform grid search, and evaluate
for name, classifier_info in classifiers.items():
    print(f"\nTraining and evaluating {name}...")
    model = classifier_info['model']
    param_grid = classifier_info['param_grid']
    
    # For classifiers that include the feature selection or scaling in a pipeline, use X_train and X_test directly
    if name == 'Neural Network':
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        grid_search.fit(X_train, y_train)
        best_clf = grid_search.best_estimator_
        y_pred = best_clf.predict(X_test)
    else:
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        grid_search.fit(X_train_selected, y_train)
        best_clf = grid_search.best_estimator_
        y_pred = best_clf.predict(X_test_selected)
    
    # Print the best parameters found by GridSearchCV
    print(f"Best parameters for {name}: {grid_search.best_params_}")
    
    # Evaluate the model
    print(f"Classification Report for {name}:")
    print(classification_report(y_test, y_pred))
    
    # For models that provide feature importances, display them
    if hasattr(best_clf, 'feature_importances_'):
        important_features = pd.DataFrame({
            'Feature': selected_feature_names,
            'Importance': best_clf.feature_importances_
        }).sort_values(by='Importance', ascending=False)
        print(f"Important features for {name}:")
        print(important_features)
    elif hasattr(best_clf, 'coef_'):
        # For linear models like Logistic Regression
        importance = np.abs(best_clf.coef_[0])
        important_features = pd.DataFrame({
            'Feature': selected_feature_names,
            'Importance': importance
        }).sort_values(by='Importance', ascending=False)
        print(f"Important features for {name}:")
        print(important_features)
    else:
        print(f"{name} does not provide feature importances directly.")


Feature Extraction: 100%|██████████| 14/14 [00:02<00:00,  6.69it/s]
c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [ 3 13 23] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw



Training and evaluating Random Forest...
Best parameters for Random Forest: {'max_depth': None, 'min_samples_split': 10, 'n_estimators': 500}
Classification Report for Random Forest:
              precision    recall  f1-score   support

           0       0.67      0.67      0.67         6
           1       0.67      0.67      0.67         6

    accuracy                           0.67        12
   macro avg       0.67      0.67      0.67        12
weighted avg       0.67      0.67      0.67        12

Important features for Random Forest:
                  Feature  Importance
8    O2__absolute_maximum    0.231114
0          Oz__sum_values    0.165242
7             O2__maximum    0.147457
2          O2__sum_values    0.136516
9             O2__minimum    0.133236
4  O2__standard_deviation    0.104246
6    O2__root_mean_square    0.082190
1                Oz__mean    0.000000
3                O2__mean    0.000000
5            O2__variance    0.000000

Training and evaluating Logistic

c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

Best parameters for Support Vector Machine: {'C': 100, 'gamma': 'scale', 'kernel': 'rbf'}
Classification Report for Support Vector Machine:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.33      0.50      0.40         6

    accuracy                           0.25        12
   macro avg       0.17      0.25      0.20        12
weighted avg       0.17      0.25      0.20        12

Support Vector Machine does not provide feature importances directly.

Training and evaluating Gradient Boosting...
Best parameters for Gradient Boosting: {'learning_rate': 0.1, 'max_depth': 3, 'min_samples_split': 5, 'n_estimators': 100}
Classification Report for Gradient Boosting:
              precision    recall  f1-score   support

           0       0.67      0.67      0.67         6
           1       0.67      0.67      0.67         6

    accuracy                           0.67        12
   macro avg       0.67      0.

c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Best parameters for Neural Network: {'classifier__activation': 'relu', 'classifier__alpha': 0.0001, 'classifier__hidden_layer_sizes': (50,), 'classifier__learning_rate': 'constant', 'classifier__solver': 'sgd'}
Classification Report for Neural Network:
              precision    recall  f1-score   support

           0       0.60      0.50      0.55         6
           1       0.57      0.67      0.62         6

    accuracy                           0.58        12
   macro avg       0.59      0.58      0.58        12
weighted avg       0.59      0.58      0.58        12

Neural Network does not provide feature importances directly.

Training and evaluating XGBoost...
Best parameters for XGBoost: {'colsample_bytree': 0.7, 'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.7}
Classification Report for XGBoost:
              precision    recall  f1-score   support

           0       0.71      0.83      0.77         6
           1       0.80      0.67      0.73   

c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
c:\Users\WERPELGA\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\core.py:158: UserWarning: [19:56:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [35]:
# Assuming 'best_clf' is your best XGBoost classifier from GridSearchCV
# if name == 'XGBoost':
#     # Save the best XGBoost model using joblib
#     joblib.dump(best_clf, 'best_xgboost_model.pkl')
#     print("XGBoost model saved to 'best_xgboost_model.pkl'")


XGBoost model saved to 'best_xgboost_model.pkl'


In [ ]:
# import joblib

# # Save the trained classifier
# joblib.dump(best_clf, 'trained_random_forest.pkl')

# # Save the feature selector
# joblib.dump(selector, 'feature_selector.pkl')
